# PyCity Schools Analysis

- School Size Impact: Smaller schools (<1000 students) generally outperform larger ones in both math and reading scores.

- Spending Per Student: Higher spending per student doesn't necessarily translate to better performance. Some schools with lower spending achieve higher scores.

- School Type Influence: Charter schools often have better average scores and passing rates compared to district schools.

- Reading vs. Math: Reading scores are slightly higher than math scores across schools, indicating a potential area of focus for improvement in math.
  
---

In [1111]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [1112]:
# Calculate the total number of unique schools
school_count = school_data_complete['school_name'].nunique()
school_count

15

In [1113]:
# Calculate the total number of students
student_count = school_data_complete['student_name'].count() 
student_count

39170

In [1114]:
# Calculate the total budget
total_budget = school_data_complete.groupby('school_name')['budget'].unique().sum()[0]
total_budget

24649428

In [1115]:
# Calculate the average (mean) math score
average_math_score = school_data_complete['math_score'].mean()

In [1116]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete['reading_score'].mean()
average_math_score

78.98537145774827

In [1117]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [1118]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [1119]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [1120]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({
    'Total Schools': [school_count],
    'Total Students': [student_count],
    'Total Budget': [total_budget],
    'Average Math Score': [average_math_score],
    'Average Reading Score': [average_reading_score],
    '% Passing Math': [passing_math_percentage],
    '% Passing Reading': [passing_reading_percentage],
    '% Overall Passing Rate': [overall_passing_rate]
})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{:.2f}%".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{:.2f}%".format)
district_summary["% Overall Passing Rate"] = district_summary["% Overall Passing Rate"].map("{:.2f}%".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:.2f}".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

In [1121]:
# Use the code provided to select all of the school types
school_types = (school_data_complete.drop_duplicates('school_name')).set_index('school_name')['type']
school_types

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

In [1122]:
# Calculate the total student count per school
per_school_counts = (school_data_complete.drop_duplicates('school_name')).set_index('school_name')['size']
per_school_counts

school_name
Huang High School        2917
Figueroa High School     2949
Shelton High School      1761
Hernandez High School    4635
Griffin High School      1468
Wilson High School       2283
Cabrera High School      1858
Bailey High School       4976
Holden High School        427
Pena High School          962
Wright High School       1800
Rodriguez High School    3999
Johnson High School      4761
Ford High School         2739
Thomas High School       1635
Name: size, dtype: int64

In [1123]:
# Calculate the total school budget and per capita spending per school
    #Total School Budget
total_school_budget = school_data_complete.drop_duplicates('school_name')['budget'].sum()
    #Per School Budget
per_school_budget = (school_data_complete.drop_duplicates('school_name').set_index('school_name').groupby('school_name')['budget'].sum())
    #Per School Capita
per_school_capita = per_school_budget / per_school_counts

In [1124]:
# Calculate the average math score per school
per_school_math = school_data_complete.groupby('school_name')['math_score'].mean()
# Calculate the average reading score per school
per_school_reading = school_data_complete.groupby('school_name')['reading_score'].mean()

In [1125]:
# Calculate the number of students per school with math scores of 70 or higher
#Combined Total Passing Math Greater Than 70
total_students_passing_math = (school_data_complete[school_data_complete["math_score"] >= 70].groupby('school_name')['Student ID'].count().sum())
#Total Passing Math Greater Than 70 By Schools
students_passing_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby('school_name')['Student ID'].count()
#Percent Passing Math By School
percent_school_passing_math = (students_passing_math / per_school_counts) * 100

In [1151]:
# Calculate the number of students per school with reading scores of 70 or higher
#Combined Total Passing Reading Greater Than 70
total_students_passing_reading = (school_data_complete[school_data_complete["reading_score"] >= 70].groupby('school_name')['Student ID'].count().sum())
#Total Passing REading Greater Than 70 By Schools
students_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70].groupby('school_name')['Student ID'].count()
#Percent Passing Reading By School
percent_school_passing_reading = (students_passing_reading / per_school_counts) * 100

In [1152]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()

In [1153]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100

In [1154]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({
    'School Type': school_types,
    'Total Students': per_school_counts,
    'Total School Budget': per_school_budget,
    'Per Student Budget': per_school_capita,
    'Average Math Score': per_school_math,
    '# of Students Passing Math': school_students_passing_math,
    '% Students Passing Math':per_school_passing_math,
    'Average Reading Score': per_school_reading,
    '# of Students Passing Reading': school_students_passing_reading,
    '% Students Passing Reading': per_school_passing_reading,
    '% Overall Passing Rate': overall_passing_rate
})

per_school_summary2 = per_school_summary.copy()

# Formatting
per_school_summary2['Total School Budget'] = per_school_summary2['Total School Budget'].map("${:,.2f}".format)
per_school_summary2['Per Student Budget'] = per_school_summary2['Per Student Budget'].map("${:,.2f}".format)
per_school_summary2['Average Math Score'] = per_school_summary2['Average Math Score'].map("{:.2f}".format)
per_school_summary2['Total Students'] = per_school_summary2['Total Students'].apply(lambda x: f'{x:,}')
per_school_summary2['# of Students Passing Math'] = per_school_summary2['# of Students Passing Math'].apply(lambda x: f'{x:,}')
per_school_summary2['% Students Passing Math'] = per_school_summary2['% Students Passing Math'].map("{:.2f}%".format)
per_school_summary2['Average Reading Score'] = per_school_summary2['Average Reading Score'].map("{:.2f}".format)
per_school_summary2['# of Students Passing Reading'] = per_school_summary2['# of Students Passing Reading'].apply(lambda x: f'{x:,}')
per_school_summary2['% Students Passing Reading'] = per_school_summary2['% Students Passing Reading'].map("{:.2f}%".format)
per_school_summary2['% Overall Passing Rate'] = per_school_summary2['% Overall Passing Rate'].map("{:.2f}%".format)

# Rename the index name of the DataFrame
per_school_summary2.index.name = "School Name"

# Display the DataFrame
per_school_summary2

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,# of Students Passing Math,% Students Passing Math,Average Reading Score,# of Students Passing Reading,% Students Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,"3,318",66.68%,81.03,"4,077",81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,"1,749",94.13%,83.98,"1,803",97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,"1,946",65.99%,81.16,"2,381",80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,"1,871",68.31%,80.75,"2,172",79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,"1,371",93.39%,83.82,"1,426",97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,"3,094",66.75%,80.93,"3,748",80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,395,92.51%,83.81,411,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,"1,916",65.68%,81.18,"2,372",81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,"3,145",66.06%,80.97,"3,867",81.22%,53.54%


## Highest-Performing Schools (by % Overall Passing)

In [1155]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(by='% Overall Passing Rate', ascending=False)

# Formatting
top_schools['Total School Budget'] = top_schools['Total School Budget'].map("${:,.2f}".format)
top_schools['Per Student Budget'] = top_schools['Per Student Budget'].map("${:,.2f}".format)
top_schools['Average Math Score'] = top_schools['Average Math Score'].map("{:.2f}".format)
top_schools['Total Students'] = top_schools['Total Students'].apply(lambda x: f'{x:,}')
top_schools['# of Students Passing Math'] = top_schools['# of Students Passing Math'].apply(lambda x: f'{x:,}')
top_schools['% Students Passing Math'] = top_schools['% Students Passing Math'].map("{:.2f}%".format)
top_schools['Average Reading Score'] = top_schools['Average Reading Score'].map("{:.2f}".format)
top_schools['# of Students Passing Reading'] = top_schools['# of Students Passing Reading'].apply(lambda x: f'{x:,}')
top_schools['% Students Passing Reading'] = top_schools['% Students Passing Reading'].map("{:.2f}%".format)
top_schools['% Overall Passing Rate'] = top_schools['% Overall Passing Rate'].map("{:.2f}%".format)

top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,# of Students Passing Math,% Students Passing Math,Average Reading Score,# of Students Passing Reading,% Students Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,"1,749",94.13%,83.98,"1,803",97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,"1,525",93.27%,83.85,"1,591",97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,"1,371",93.39%,83.82,"1,426",97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,"2,143",93.87%,83.99,"2,204",96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,910,94.59%,84.04,923,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

In [1156]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(by='% Overall Passing Rate', ascending=True)

# Formatting
bottom_schools['Total School Budget'] = bottom_schools['Total School Budget'].map("${:,.2f}".format)
bottom_schools['Per Student Budget'] = bottom_schools['Per Student Budget'].map("${:,.2f}".format)
bottom_schools['Average Math Score'] = bottom_schools['Average Math Score'].map("{:.2f}".format)
bottom_schools['Total Students'] = bottom_schools['Total Students'].apply(lambda x: f'{x:,}')
bottom_schools['# of Students Passing Math'] = bottom_schools['# of Students Passing Math'].apply(lambda x: f'{x:,}')
bottom_schools['% Students Passing Math'] = bottom_schools['% Students Passing Math'].map("{:.2f}%".format)
bottom_schools['Average Reading Score'] = bottom_schools['Average Reading Score'].map("{:.2f}".format)
bottom_schools['# of Students Passing Reading'] = bottom_schools['# of Students Passing Reading'].apply(lambda x: f'{x:,}')
bottom_schools['% Students Passing Reading'] = bottom_schools['% Students Passing Reading'].map("{:.2f}%".format)
bottom_schools['% Overall Passing Rate'] = bottom_schools['% Overall Passing Rate'].map("{:.2f}%".format)

bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,# of Students Passing Math,% Students Passing Math,Average Reading Score,# of Students Passing Reading,% Students Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,"2,654",66.37%,80.74,"3,208",80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,"1,946",65.99%,81.16,"2,381",80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,"1,916",65.68%,81.18,"2,372",81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,"3,094",66.75%,80.93,"3,748",80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,"3,145",66.06%,80.97,"3,867",81.22%,53.54%


## Math Scores by Grade

In [1157]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = ninth_graders.groupby('school_name')['math_score'].mean()
tenth_grader_math_scores = tenth_graders.groupby('school_name')['math_score'].mean()
eleventh_grader_math_scores = eleventh_graders.groupby('school_name')['math_score'].mean()
twelfth_grader_math_scores = twelfth_graders.groupby('school_name')['math_score'].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.merge(ninth_grade_math_scores, tenth_grader_math_scores, on='school_name', how='outer', suffixes=('9th', '10th'))
math_scores_by_grade = pd.merge(math_scores_by_grade, eleventh_grader_math_scores, on='school_name', how='outer', suffixes=('', '11th'))
math_scores_by_grade = pd.merge(math_scores_by_grade, twelfth_grader_math_scores, on='school_name', how='outer', suffixes=('', '12th'))

#Column Renaming
math_scores_by_grade = math_scores_by_grade.rename(columns={'math_score9th':'9th Grade Math Scores',
                                                            'math_score10th':'10th Grade Math Scores',
                                                            'math_score': '11th Grade Math Scores',
                                                           'math_score12th':'12th Grade Math Scores'})

#Header Formting
math_scores_by_grade['9th Grade Math Scores'] = math_scores_by_grade['9th Grade Math Scores'].map("{:.2f}".format)
math_scores_by_grade['10th Grade Math Scores'] = math_scores_by_grade['10th Grade Math Scores'].map("{:.2f}".format)
math_scores_by_grade['11th Grade Math Scores'] = math_scores_by_grade['11th Grade Math Scores'].map("{:.2f}".format)
math_scores_by_grade['12th Grade Math Scores'] = math_scores_by_grade['12th Grade Math Scores'].map("{:.2f}".format)

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade


,9th Grade Math Scores,10th Grade Math Scores,11th Grade Math Scores,12th Grade Math Scores
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

In [1158]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = ninth_graders.groupby('school_name')['reading_score'].mean()
tenth_grader_reading_scores = tenth_graders.groupby('school_name')['reading_score'].mean()
eleventh_grader_reading_scores = eleventh_graders.groupby('school_name')['reading_score'].mean()
twelfth_grader_reading_scores = twelfth_graders.groupby('school_name')['reading_score'].mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.merge(ninth_grade_reading_scores, tenth_grader_reading_scores, on='school_name', how='outer', suffixes=('9th', '10th'))
reading_scores_by_grade = pd.merge(reading_scores_by_grade, eleventh_grader_reading_scores, on='school_name', how='outer', suffixes=('', '11th'))
reading_scores_by_grade = pd.merge(reading_scores_by_grade, twelfth_grader_reading_scores, on='school_name', how='outer', suffixes=('', '12th'))

#Column Renaming
reading_scores_by_grade = reading_scores_by_grade.rename(columns={'reading_score9th':'9th Grade Reading Scores',
                                                            'reading_score10th':'10th Grade Reading Scores',
                                                            'reading_score':'11th Grade Reading Scores',
                                                           'reading_score12th':'12th Grade Reading Scores'})

#Header Formating
reading_scores_by_grade['9th Grade Reading Scores'] = reading_scores_by_grade['9th Grade Reading Scores'].map("{:.2f}".format)
reading_scores_by_grade['10th Grade Reading Scores'] = reading_scores_by_grade['10th Grade Reading Scores'].map("{:.2f}".format)
reading_scores_by_grade['11th Grade Reading Scores'] = reading_scores_by_grade['11th Grade Reading Scores'].map("{:.2f}".format)
reading_scores_by_grade['12th Grade Reading Scores'] = reading_scores_by_grade['12th Grade Reading Scores'].map("{:.2f}".format)

# Minor data wrangling
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade


,9th Grade Reading Scores,10th Grade Reading Scores,11th Grade Reading Scores,12th Grade Reading Scores
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

In [1159]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [1160]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [1161]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], bins=spending_bins, labels=labels, right=False)
school_spending_df2 = school_spending_df.copy()

# Formatting
school_spending_df2['Total School Budget'] = school_spending_df['Total School Budget'].map("${:,.2f}".format)
school_spending_df2['Per Student Budget'] = school_spending_df['Per Student Budget'].map("${:,.2f}".format)
school_spending_df2['Average Math Score'] = school_spending_df['Average Math Score'].map("{:.2f}".format)
school_spending_df2['Total Students'] = school_spending_df['Total Students'].apply(lambda x: f'{x:,}')
school_spending_df2['# of Students Passing Math'] = school_spending_df['# of Students Passing Math'].apply(lambda x: f'{x:,}')
school_spending_df2['% Students Passing Math'] = school_spending_df['% Students Passing Math'].map("{:.2f}%".format)
school_spending_df2['Average Reading Score'] = school_spending_df['Average Reading Score'].map("{:.2f}".format)
school_spending_df2['# of Students Passing Reading'] = school_spending_df['# of Students Passing Reading'].apply(lambda x: f'{x:,}')
school_spending_df2['% Students Passing Reading'] = school_spending_df['% Students Passing Reading'].map("{:.2f}%".format)
school_spending_df2['% Overall Passing Rate'] = school_spending_df['% Overall Passing Rate'].map("{:.2f}%".format)

school_spending_df2

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,# of Students Passing Math,% Students Passing Math,Average Reading Score,# of Students Passing Reading,% Students Passing Reading,% Overall Passing Rate,Spending Ranges (Per Student)
school_name,,,,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,"3,318",66.68%,81.03,"4,077",81.93%,54.64%,$585-630
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,"1,749",94.13%,83.98,"1,803",97.04%,91.33%,<$585
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,"1,946",65.99%,81.16,"2,381",80.74%,53.20%,$630-645
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,"1,871",68.31%,80.75,"2,172",79.30%,54.29%,$630-645
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,"1,371",93.39%,83.82,"1,426",97.14%,90.60%,$585-630
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,"3,094",66.75%,80.93,"3,748",80.86%,53.53%,$645-680
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,395,92.51%,83.81,411,96.25%,89.23%,<$585
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,"1,916",65.68%,81.18,"2,372",81.32%,53.51%,$645-680
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,"3,145",66.06%,80.97,"3,867",81.22%,53.54%,$645-680


In [1162]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Students Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Students Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing Rate"].mean()


In [1163]:
# Assemble into DataFrame
spending_summary = pd.merge(spending_math_scores, spending_reading_scores,  on='Spending Ranges (Per Student)', how='left')
spending_summary = pd.merge(spending_summary, spending_passing_math,  on='Spending Ranges (Per Student)', how='left')
spending_summary = pd.merge(spending_summary, spending_passing_reading,  on='Spending Ranges (Per Student)', how='left')
spending_summary = pd.merge(spending_summary, overall_passing_spending,  on='Spending Ranges (Per Student)', how='left')

# Formatting
spending_summary['Average Math Score'] = spending_summary['Average Math Score'].map("{:.2f}".format)
spending_summary['% Students Passing Math'] = spending_summary['% Students Passing Math'].map("{:.2f}%".format)
spending_summary['Average Reading Score'] = spending_summary['Average Reading Score'].map("{:.2f}".format)
spending_summary['% Students Passing Reading'] = spending_summary['% Students Passing Reading'].map("{:.2f}%".format)
spending_summary['% Overall Passing Rate'] = spending_summary['% Overall Passing Rate'].map("{:.2f}%".format)

# Display results
spending_summary

,Average Math Score,Average Reading Score,% Students Passing Math,% Students Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46%,96.61%,90.37%
$585-630,81.90,83.16,87.13%,92.72%,81.42%
$630-645,78.52,81.62,73.48%,84.39%,62.86%
$645-680,77.00,81.03,66.16%,81.13%,53.53%


## Scores by School Size

In [1170]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [1171]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["Total Students"] = pd.cut(per_school_summary["Total Students"], bins=size_bins, labels=labels, right=False)

per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,# of Students Passing Math,% Students Passing Math,Average Reading Score,# of Students Passing Reading,% Students Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,,,
Bailey High School,District,Large (2000-5000),3124928,628.0,77.048432,3318,66.680064,81.033963,4077,81.933280,54.642283
Cabrera High School,Charter,Medium (1000-2000),1081356,582.0,83.061895,1749,94.133477,83.975780,1803,97.039828,91.334769
Figueroa High School,District,Large (2000-5000),1884411,639.0,76.711767,1946,65.988471,81.158020,2381,80.739234,53.204476
Ford High School,District,Large (2000-5000),1763916,644.0,77.102592,1871,68.309602,80.746258,2172,79.299014,54.289887
Griffin High School,Charter,Medium (1000-2000),917500,625.0,83.351499,1371,93.392371,83.816757,1426,97.138965,90.599455
Hernandez High School,District,Large (2000-5000),3022020,652.0,77.289752,3094,66.752967,80.934412,3748,80.862999,53.527508
Holden High School,Charter,Small (<1000),248087,581.0,83.803279,395,92.505855,83.814988,411,96.252927,89.227166
Huang High School,District,Large (2000-5000),1910635,655.0,76.629414,1916,65.683922,81.182722,2372,81.316421,53.513884
Johnson High School,District,Large (2000-5000),3094650,650.0,77.072464,3145,66.057551,80.966394,3867,81.222432,53.539172


In [1172]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["Total Students"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["Total Students"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["Total Students"])["% Students Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["Total Students"])["% Students Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["Total Students"])["% Overall Passing Rate"].mean()

In [1173]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary =  pd.DataFrame({
    "Average Math Score": size_math_scores,
    "Average Reading Score": size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing Rate": size_overall_passing
})
#reading_scores_by_grade = pd.merge(ninth_grade_reading_scores, tenth_grader_reading_scores, on='school_name', how='outer', suffixes=('9th', '10th'))

# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Total Students,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [1174]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Students Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Students Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing Rate"].mean()

In [1175]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({
    "Average Math Score": average_math_score_by_type,
    "Average Reading Score": average_reading_score_by_type,
    "% Students Passing Math": average_percent_passing_math_by_type,
    "% Students Passing Reading": average_percent_passing_reading_by_type,
    "% Overall Passing Rate": average_percent_overall_passing_by_type
})

# Display results
type_summary

,Average Math Score,Average Reading Score,% Students Passing Math,% Students Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
